In [2]:
import numpy as np
import pandas as pd
import tqdm
import seaborn as sns
import nltk
import matplotlib.pyplot as plt
import time
import sklearn

In [3]:
data = pd.read_csv("data/abcnews-date-text.csv")

### Exploring dataset

In [5]:
data.shape

(1186018, 2)

In [6]:
data.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers
